# first exercise

We define a function which counts the number of lines for each files in specifique format in a specifique directory.
To count the number of lines for a file, we count the '\n' in the file. We read the file block by block so that it can fit in RAM.

In [1]:
import glob
def blocks(files, size=65536):
    while True:
        b = files.read(size)
        if not b:break
        yield b
        
def countLine(files):
    with open(files, 'r') as f:
        return sum(bl.count('\n') for bl in blocks(f))

def countLineForEach(directory, format_f):
    name = directory + '\\**\*.' + format_f
    for filename in glob.iglob(name, recursive=True):
        print(filename + ' :\n' + str(countLine(filename)) + '  lines')
 
directory = 'C:\\Users\\XIA\\Desktop\\amadeus_test'
format_f = 'csv'
countLineForEach(directory, format_f)

C:\Users\XIA\Desktop\amadeus_test\data\bookings.csv :
10000011  lines
C:\Users\XIA\Desktop\amadeus_test\data\searches.csv :
20390198  lines


# second exercise

The data can not be read entirely in my computer (the memory of my current computer is relatively small), so I have to process the data chunk by chunk and combine the result of each chunk to get the final result.

This may be verbose.

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
###set the maximum number of columns that can be shown
pd.set_option('display.max_columns', 100)

nb_lines_booking = 10000011  
nb_lines_searches = 20390198  
file_dir = 'C:\\Users\\XIA\\Desktop\\amadeus_test\\data\\bookings.csv'
data_piece = pd.read_csv(file_dir, nrows = 1, engine='python', sep='^')
data_piece.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL


In [3]:
chunksize = 10**6
data_booking = pd.read_csv(file_dir, sep='^', chunksize=chunksize)
res = None
n = 10
year = 2013
for chunk in data_booking:
    chunk = chunk[chunk['year'] == year]
    pax_by_chunk = chunk.groupby(['arr_port','arr_city']).agg(np.sum)['pax']
    if(res is None):
        res = Counter(pax_by_chunk.to_dict())
    else:
        res = res + res
        
top_n = res.most_common(n)
for i in range(n):
    print(top_n[i][0][0]+'  '+top_n[i][0][1]+' : '+str(top_n[i][1]))

g:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:199: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  shell.run_cell(code, store_history=store_history, silent=silent)


LHR       LON      : 9094144
MCO       ORL      : 7263232
LAX       LAX      : 7222272
LAS       LAS      : 7130112
JFK       NYC      : 6786048
CDG       PAR      : 6603776
BKK       BKK      : 6088704
MIA       MIA      : 5954560
SFO       SFO      : 5939200
DXB       DXB      : 5692416


# Third exercise